In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [2]:
data=pd.read_csv('/kaggle/input/loan-data-set/loan_data_set.csv')

In [3]:
df=pd.DataFrame(data)
df.head()


,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [4]:
df.shape


(614, 13)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            614 non-null    object 
 1   Gender             601 non-null    object 
 2   Married            611 non-null    object 
 3   Dependents         599 non-null    object 
 4   Education          614 non-null    object 
 5   Self_Employed      582 non-null    object 
 6   ApplicantIncome    614 non-null    int64  
 7   CoapplicantIncome  614 non-null    float64
 8   LoanAmount         592 non-null    float64
 9   Loan_Amount_Term   600 non-null    float64
 10  Credit_History     564 non-null    float64
 11  Property_Area      614 non-null    object 
 12  Loan_Status        614 non-null    object 
dtypes: float64(4), int64(1), object(8)
memory usage: 62.5+ KB


In [6]:
df.isnull().sum()

Loan_ID               0
Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [7]:
df.describe()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History
count,614.000000,614.000000,592.000000,600.00000,564.000000
mean,5403.459283,1621.245798,146.412162,342.00000,0.842199
std,6109.041673,2926.248369,85.587325,65.12041,0.364878
min,150.000000,0.000000,9.000000,12.00000,0.000000
25%,2877.500000,0.000000,100.000000,360.00000,1.000000
50%,3812.500000,1188.500000,128.000000,360.00000,1.000000
75%,5795.000000,2297.250000,168.000000,360.00000,1.000000
max,81000.000000,41667.000000,700.000000,480.00000,1.000000


since Loan_ID has no relecance to the model. we will drop it

In [8]:
df = df.drop('Loan_ID', axis=1)

In [9]:
features=['Gender', 'Married', 'Education', 'Self_Employed', 'Property_Area','Loan_Status']
for features in features:
    count=df[features].value_counts()
    print(f"value count of {features}:{count} \n")

value count of Gender:Gender
Male      489
Female    112
Name: count, dtype: int64 

value count of Married:Married
Yes    398
No     213
Name: count, dtype: int64 

value count of Education:Education
Graduate        480
Not Graduate    134
Name: count, dtype: int64 

value count of Self_Employed:Self_Employed
No     500
Yes     82
Name: count, dtype: int64 

value count of Property_Area:Property_Area
Semiurban    233
Urban        202
Rural        179
Name: count, dtype: int64 

value count of Loan_Status:Loan_Status
Y    422
N    192
Name: count, dtype: int64 



# Categorical data is to be filled with mode
## Gender,Self_employed,credit history,Dependent  

In [10]:
mode_Gender = df['Gender'].mode()
df['Gender'].fillna(mode_Gender[0],inplace=True)

mode_Married = df['Married'].mode()
df['Married'].fillna(mode_Married[0],inplace=True)

mode_Self_Employed = df['Self_Employed'].mode()
df['Self_Employed'].fillna(mode_Self_Employed[0],inplace=True)

mode_Credit_History = df['Credit_History'].mode()
df['Credit_History'].fillna(mode_Credit_History[0],inplace=True)

mode_Dependents = df['Dependents'].mode()
df['Dependents'].fillna(mode_Dependents[0],inplace=True)


In [11]:
# Replace '3+' with 3 in Dependent Column
df['Dependents'] = df['Dependents'].replace('3+', 3)
# If the column is of type object or string, you might want to convert it to numeric type (int) after replacement
df['Dependents'] = df['Dependents'].astype(int)
df['Credit_History']=df['Credit_History'].astype('object')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Gender             614 non-null    object 
 1   Married            614 non-null    object 
 2   Dependents         614 non-null    int64  
 3   Education          614 non-null    object 
 4   Self_Employed      614 non-null    object 
 5   ApplicantIncome    614 non-null    int64  
 6   CoapplicantIncome  614 non-null    float64
 7   LoanAmount         592 non-null    float64
 8   Loan_Amount_Term   600 non-null    float64
 9   Credit_History     614 non-null    object 
 10  Property_Area      614 non-null    object 
 11  Loan_Status        614 non-null    object 
dtypes: float64(3), int64(2), object(7)
memory usage: 57.7+ KB


# filling Numerical data with Median


In [12]:
median_Loan_Amount = df['LoanAmount'].median()
df['LoanAmount'].fillna(median_Loan_Amount,inplace=True)

median_Loan_Amount_Term = df['Loan_Amount_Term'].median()
df['Loan_Amount_Term'].fillna(median_Loan_Amount_Term,inplace=True)

In [13]:
df.isnull().sum()

Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

# NO MORE NULL VALUES
## Outliers < Q1−1.5×IQR   Outliers>Q3+1.5×IQR

In [14]:
# Q1=2877(25%) Q3=5795(75%)
# IQR=5729-2877 = 2918
# UPPER BOUND= Q1-1.5xIQR
#            = 2877- 1.5(2918)
#            =-1500
# LOWER BOUND= Q3+1.5xIQR
#            = 5795+1.5(2918)
#            =10172

# Any values <−1500 or >10172 are considered outliers.

In [15]:
# Columns to process
columns_to_clean = ['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term']

# Remove outliers directly from the original DataFrame
for column in columns_to_clean:
    # Calculate Q1, Q3, and IQR
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1

    # Define lower and upper bounds
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Filter the DataFrame to keep only rows within the bounds
    df = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

# Display the modified DataFrame
print("Data after removing outliers directly from the original DataFrame:")
df.describe()


Data after removing outliers directly from the original DataFrame:


,Dependents,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term
count,445.000000,445.000000,445.000000,445.000000,445.0
mean,0.674157,3968.060674,1326.714876,125.170787,360.0
std,0.972311,1759.700146,1376.571512,38.957965,0.0
min,0.000000,150.000000,0.000000,26.000000,360.0
25%,0.000000,2718.000000,0.000000,100.000000,360.0
50%,0.000000,3588.000000,1330.000000,124.000000,360.0
75%,1.000000,4755.000000,2200.000000,150.000000,360.0
max,3.000000,10000.000000,5625.000000,236.000000,360.0


#                                            ENCODING
## only done on non-numerical data
## using one-hot encoding to avoid multicolinearity

In [16]:
from sklearn.preprocessing import LabelEncoder

# List of columns to apply label encoding
columns_to_encode = ['Gender', 'Married', 'Education', 'Self_Employed', 'Property_Area', 'Loan_Status']

# Initialize the label encoder
label_encoder = LabelEncoder()

# Apply label encoding to each specified column
for col in columns_to_encode:
    df[col] = label_encoder.fit_transform(df[col])

# Display the updated DataFrame
print("DataFrame after label encoding:")
print(df.head())


DataFrame after label encoding:
   Gender  Married  Dependents  Education  Self_Employed  ApplicantIncome  \
0       1        0           0          0              0             5849   
1       1        1           1          0              0             4583   
2       1        1           0          0              1             3000   
3       1        1           0          1              0             2583   
4       1        0           0          0              0             6000   

   CoapplicantIncome  LoanAmount  Loan_Amount_Term Credit_History  \
0                0.0       128.0             360.0            1.0   
1             1508.0       128.0             360.0            1.0   
2                0.0        66.0             360.0            1.0   
3             2358.0       120.0             360.0            1.0   
4                0.0       141.0             360.0            1.0   

   Property_Area  Loan_Status  
0              2            1  
1              0          

In [17]:
df.head()


,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,1,0,0,0,0,5849,0.0,128.0,360.0,1.0,2,1
1,1,1,1,0,0,4583,1508.0,128.0,360.0,1.0,0,0
2,1,1,0,0,1,3000,0.0,66.0,360.0,1.0,2,1
3,1,1,0,1,0,2583,2358.0,120.0,360.0,1.0,2,1
4,1,0,0,0,0,6000,0.0,141.0,360.0,1.0,2,1


## Splitting data for training and testing

In [18]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

X = df.drop('Loan_Status', axis=1)  # Features
y = df['Loan_Status']              # Target variable
print("Shape of y:", y.shape)  # Check shape of y before splitting


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print("Shape of y_train:", y_train.shape)  # Check shape of y_train
print("Shape of y_test:", y_test.shape)    # Check shape of y_test



Shape of y: (445,)
Shape of y_train: (356,)
Shape of y_test: (89,)


In [19]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
X_train_scaled=scaler.fit_transform(X_train)
X_test_scaled=scaler.fit_transform(X_test)

In [20]:
lr=LogisticRegression()

In [21]:
lr=LogisticRegression()
lr.fit(X_train,y_train)

LogisticRegression()

In [22]:
y_pred=lr.predict(X_test)

In [23]:
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Print evaluation metrics
print("Model Accuracy:", accuracy)
print("\nConfusion Matrix:\n", conf_matrix)
print("\nClassification Report:\n", class_report)

Model Accuracy: 0.8764044943820225

Confusion Matrix:
 [[10 11]
 [ 0 68]]

Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.48      0.65        21
           1       0.86      1.00      0.93        68

    accuracy                           0.88        89
   macro avg       0.93      0.74      0.79        89
weighted avg       0.89      0.88      0.86        89

